# Find a Feasible Mobile Manipulation (from Real Data)

In [1]:
%matplotlib widget

import numpy as np
import feasibility_map as fmap

## 1. Offline

### 1.0. Utils

### 1.1. Configuration

1. `Rsize`: the radius of the robot-base
2. `M`: X-Y-Cr manipulability map

```python
raw:
    [[Cr x y manip],
     [Cr x y manip], ...]
layers:
    [deg]: [[x, ...],
            [y, ...],
            [manip, ...]]
```

In [2]:
Rsize = 0.3

- npy data
    - Cr (radian)
    - x (m)
    - y (m)
    - manip (no unit)

In [3]:
# In this file, Cr is radians.
M = np.load('real_manip.npy')
# So, convert it to degrees.
M[:,0] = np.round(np.degrees(M[:,0]), decimals=3)
print(M.shape)
print(M)

(893, 4)
[[ 0.00000e+00  1.00000e-01 -4.40000e-01  1.51625e-02]
 [ 1.00000e+01  1.00000e-01 -4.40000e-01  1.51625e-02]
 [ 2.00000e+01  1.00000e-01 -4.40000e-01  1.51625e-02]
 ...
 [ 6.00000e+01  2.80000e-01 -8.00000e-02  8.97977e-03]
 [ 7.00000e+01  2.80000e-01 -8.00000e-02  8.97977e-03]
 [ 8.00000e+01  2.80000e-01 -8.00000e-02  8.97977e-03]]


### 1.2. Preparation

1. Convert the `M` to a feasibility map `F` (helical shape).

Cr, x, y, manipulability

In [4]:
F = fmap.FeasibilityMap(M, Rsize, is_jupyter=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 99)
(3, 99)
(893, 4)


## 2. Online

### 2.1. Input

Known
1. `self.feasi_layers`: feasibility map
2. `self.robot_radius`: robot base (circle radius)

---
Input

1. `Pt`: Position of the target object (relative to the robot’s current pose) => `(x, y)`
2. `Obs`: Area list of ground obstacles => `[CollisionModel, ...]`
3. `Cr`: Constraints on the approach angle (relative to the robot heading) (-90 ~ 90) => `(min, max)`
4. `Ct`: Constraints on the approach angle (relative to the target heading) (-180 ~ 180) => `(min, max)`

In [5]:
from collision import CollisionBox

Pt = (0.0, 0.0)
Obs = [CollisionBox((0.2, 0.0), np.radians(0.0), 0.45, 1.0)]
Cr = (30, 30)  # min, max
Ct = (-180, 180) # min, max

### 2.2. Process

1. Cut the range of `Cr` from `F` and set it to `Fcut`.
2. ~~[SKIP] Scan the maximum points for the radius and angle by each circle in `Fcut`.~~
3. And extract only the maximum as a `Fmax`.
4. Wipe the `Fmax` in the range of `Ct`.
5. Remove all obstacle areas from `Fwiped` with the offset of `Rsize`.

In [6]:
F.calc(Pt, Obs, Cr, Ct, (0.05, 0.5, 0.02))

Cut the range of `Cr` from `Fraw` and set it to `Fcut`.
min Cr:  30
max Cr:  30


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

And extract only the maximum as a `Fmax`.
sq_sections.shape:  (23,)
Fmax.shape:  (19, 4)
Fmax:
 [[ 3.00000000e+01 -7.66025409e-02  6.73205107e-02  4.30307016e-02]
 [ 3.00000000e+01 -6.16025403e-02  9.33012739e-02  4.04035002e-02]
 [ 3.00000000e+01 -1.02583304e-01  8.23205113e-02  4.20637988e-02]
 [ 3.00000000e+01 -1.13564067e-01  1.23301268e-01  4.07138988e-02]
 [ 3.00000000e+01 -9.85640660e-02  1.49282038e-01  3.97801995e-02]
 [ 3.00000000e+01 -1.24544829e-01  1.64282039e-01  3.89292017e-02]
 [ 3.00000000e+01 -1.09544829e-01  1.90262794e-01  3.86510007e-02]
 [ 3.00000000e+01 -9.45448279e-02  2.16243550e-01  3.77761014e-02]
 [ 3.00000000e+01 -7.95448273e-02  2.42224321e-01  3.72181013e-02]
 [ 3.00000000e+01  4.33974601e-02  2.75166601e-01  4.08669002e-02]
 [ 3.00000000e+01  5.83974607e-02  3.01147372e-01  4.61509004e-02]
 [ 3.00000000e+01  3.24166976e-02  3.16147357e-01  4.83170003e-02]
 [ 3.00000000e+01  4.74166982e-02  3.42128128e-01  4.87559997e-02]
 [ 3.00000000e+01  2.14359351e-02

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Wipe the `Fmax` in the range of `Ct`. => `Fwiped`
min Ct:  -180
max Ct:  180
Fwiped.shape:  (1652, 5)
Interval is  0.02  [m].


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Remove all obstacle areas from `Fwiped` with the offset of `Rsize`. => `Fclean`
Fclean.shape:  (1652, 5)
Fclean.shape:  (164, 5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.3. Output

1. Candidate poses (sorted in descending order of manipulability)

In [7]:
candidates = F.get_candidates()

Fclean[:10]:
 [[ 7.96239403e+01  3.00000000e+01 -3.27993108e-01  1.08261351e-01
   4.87559997e-02]
 [ 8.29416045e+01  3.00000000e+01 -3.33708680e-01  8.90983517e-02
   4.87559997e-02]
 [ 8.62592687e+01  3.00000000e+01 -3.38305674e-01  6.96366992e-02
   4.87559997e-02]
 [ 8.95769328e+01  3.00000000e+01 -3.41768683e-01  4.99416276e-02
   4.87559997e-02]
 [ 9.28945970e+01  3.00000000e+01 -3.44086099e-01  3.00791538e-02
   4.87559997e-02]
 [ 9.62122612e+01  3.00000000e+01 -3.45250154e-01  1.01158561e-02
   4.87559997e-02]
 [ 9.95299254e+01  3.00000000e+01 -3.45256945e-01 -9.88134952e-03
   4.87559997e-02]
 [ 1.02847590e+02  3.00000000e+01 -3.44106450e-01 -2.98454333e-02
   4.87559997e-02]
 [ 1.06165254e+02  3.00000000e+01 -3.41802526e-01 -4.97094765e-02
   4.87559997e-02]
 [ 1.09482918e+02  3.00000000e+01 -3.38352896e-01 -6.94068958e-02
   4.87559997e-02]]
Fclean[:10] (sorted):
 [[ 7.96239403e+01  3.00000000e+01 -3.27993108e-01  1.08261351e-01
   4.87559997e-02]
 [ 1.16118246e+02  3.000000

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …